In [ ]:
pip install --upgrade shap


In [ ]:
import shap
print(shap.__version__)

0.50.0


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import  mean_squared_error,mean_absolute_error
import shap

np.random.seed(42)
tf.random.set_seed(42)

t=np.arange(0,1000)

trend=0.05*t
seasonality=10*np.sin(2*np.pi*t/50)
noise=np.random.normal(0,2,size=len(t))

value=trend+seasonality+noise

df=pd.DataFrame({
    "time":t,
    "value":value,
    "trend":trend,
    "seasonality":seasonality,
    "noise_feature":np.random.normal(0,1,len(t))
})
print(df.head())

def create_sequences(data,time_steps=1):
  x,y=[],[]
  for i in range(len(data)-time_steps):
    v=data[i:(i+time_steps)]
    x.append(v)
    y.append(data[i+time_steps])
  return np.array(x),np.array(y)

TIME_STEPS=10
#asssuming data is numpy array
x,y=create_sequences(df["value"].values,TIME_STEPS)

#Train,Test
split=int(len(x)*0.8)

x_train=x[:split]
x_test=x[split:]

y_train=y[:split]
y_test=y[split:]

# Reshape for LSTM (VERY IMPORTANT)
x_train = x_train.reshape(x_train.shape[0], TIME_STEPS, 1)
x_test  = x_test.reshape(x_test.shape[0], TIME_STEPS, 1)

#Definig the model
model=Sequential()
model.add(LSTM(units=50,activation="relu",input_shape=(TIME_STEPS,1)))
model.add(Dense(units=1))
model.compile(optimizer="adam",loss="mean_squared_error")
model.summary()

#Training the model
model.fit(x_train,y_train,epochs=20,batch_size=32,verbose=0)

y_pred=model.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")

explainer = shap.DeepExplainer(model,x_train[:100])   # background samples

#predictions
shap_values=explainer.shap_values(x_test[:20])

shap.summary_plot(shap_values[0],x_test[:20],plot_type="dot")
#barplot
shap.summary_plot(shap_values[0],x_test[:20],plot_type="bar")

plt.show()

   time     value  trend  seasonality  noise_feature
0     0  0.993428   0.00     0.000000       1.399355
1     1  1.026804   0.05     1.253332       0.924634
2     2  3.882276   0.10     2.486899       0.059630
3     3  6.877305   0.15     3.681246      -0.646937
4     4  4.549230   0.20     4.817537       0.698223


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 50)             │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,451 (40.82 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)

LookupError: gradient registry has no entry for: shap_DivNoNan